In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_2226410/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.8)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_3_6,0.998214,0.679335,0.998348,0.996129,0.997702,0.000691,0.190361,0.000762,0.000891,0.000827,0.011736,0.026278,0.996279,0.026489,144.556151,328.914475,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
1,model_1_3_8,0.998179,0.676142,0.997680,0.995238,0.996988,0.000704,0.192256,0.001070,0.001097,0.001084,0.011973,0.026536,0.996205,0.026749,144.517045,328.875369,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
2,model_1_3_4,0.998177,0.677589,0.998723,0.996814,0.998161,0.000705,0.191397,0.000589,0.000734,0.000661,0.011756,0.026548,0.996202,0.026761,144.515262,328.873586,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
3,model_1_3_7,0.998176,0.675468,0.997803,0.995823,0.997254,0.000705,0.192656,0.001013,0.000962,0.000988,0.012066,0.026554,0.996200,0.026767,144.514300,328.872624,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
4,model_1_2_29,0.998175,0.675877,0.994457,0.995591,0.996132,0.000706,0.192413,0.001400,0.001516,0.001458,0.011753,0.026564,0.996197,0.026777,144.512782,328.871106,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,model_1_0_3,0.277645,0.123442,0.269401,0.289927,0.324171,0.279245,0.520362,0.292999,0.131008,0.212004,0.299160,0.528436,-0.504907,0.532681,132.551331,316.909655,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
496,model_1_9_6,0.254929,0.241098,-2.405732,0.885267,-0.835119,0.288026,0.450517,1.138715,0.031806,0.585260,0.064213,0.536681,-0.552232,0.540992,132.489406,316.847730,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
497,model_1_0_2,0.211046,0.045575,0.167021,0.238541,0.243616,0.304991,0.566588,0.334058,0.140489,0.237273,0.316582,0.552259,-0.643654,0.556695,132.374949,316.733273,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
498,model_1_0_1,0.129593,-0.005417,0.072068,0.165109,0.161325,0.336478,0.596859,0.372137,0.154037,0.263087,0.336261,0.580068,-0.813349,0.584727,132.178443,316.536767,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
